### Respiratoy sound automatic annotation (1D Conv, without spectrograms, no downsampling)
Attempt automatic annotation of sound files in the same format as the provided text files.
Use the sound data directly without creating spectograms

In [1]:
import os
import glob
import re
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import soundfile as sf

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, cross_val_score
from sklearn.metrics import confusion_matrix, recall_score, accuracy_score, precision_score, precision_recall_curve, plot_precision_recall_curve
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

import tensorflow as tf
from tensorflow import keras

try:
    os.environ['KAGGLE_DATA_PROXY_TOKEN']
except KeyError:
    dir_out = "./"
    dir_files = "Respiratory_Sound_Database/Respiratory_Sound_Database/"
    dir_annot_csv = "./"
    fname_demo = dir_path + "demographic_info.txt"
else:
    dir_out = "/kaggle/working/"
    dir_files = "/kaggle/input/respiratory-sound-database/Respiratory_Sound_Database/Respiratory_Sound_Database/"
    dir_annot_csv = "/kaggle/input/recording-annotations/"
    fname_demo = "/kaggle/input/respiratory-sound-database/" + "demographic_info.txt"
    
fname_diag = dir_files + "patient_diagnosis.csv"
fname_annot = dir_annot_csv + "rec_annotation.csv"
dir_audio = dir_files + "audio_and_txt_files/"

In [2]:
group_pat_num = "([0-9]{3})"
group_rec_index = "([0-9][a-z][0-9])"
group_chest_loc = "(Tc|Al|Ar|Pl|Pr|Ll|Lr)"
group_acc_modes = "(sc|mc)"
group_equipments = "(AKGC417L|LittC2SE|Litt3200|Meditron)"

regex_info = re.compile("_".join([group_pat_num, group_rec_index, group_chest_loc, group_acc_modes, group_equipments]))

top = os.getcwd()
os.chdir(dir_audio)
fnames = glob.glob("*.wav")

l_wav_rec = []
dict_wav_rec = {}
min_len = np.inf
max_len = 0

for fname in fnames:
    match_info = regex_info.match(fname)
    pat_num = int(match_info.group(1))
    rec_index = match_info.group(2)
    chest_loc = match_info.group(3)
    acc_mode = match_info.group(4)
    equipment = match_info.group(5)
    
    wav_content = sf.read(fname)[0]
    l_wav_rec.append([pat_num, rec_index, chest_loc, wav_content])
    dict_wav_rec[(pat_num, rec_index, chest_loc)] = wav_content
    
    if len(wav_content) > max_len:
        max_len = len(wav_content)
        # for getting the corresponding annotation below
        max_patnum = pat_num
        max_recindex = rec_index
        max_chestloc = chest_loc
    
    if len(wav_content) < min_len:
        min_len = len(wav_content)
        # for getting the corresponding annotation below
        min_patnum = pat_num
        min_recindex = rec_index
        min_chestloc = chest_loc

os.chdir(top)

# pad all recordings to same length
for i in range(len(l_wav_rec)):
    if len(l_wav_rec[i][3]) < max_len:
        padding = [0] * ( max_len - len(l_wav_rec[i][3]) )
        l_wav_rec[i][3] = np.append(l_wav_rec[i][3], padding)

# pad all recordings to multiple of length of shortest recording
# for i in range(len(l_wav_rec)):
#     if len(l_wav_rec[i][3]) % min_len != 0:
#         padding = [0] * ( min_len - len(l_wav_rec[i][3]) % min_len)
#         l_wav_rec[i][3] = np.append(l_wav_rec[i][3], padding)

l_wav_rec.sort(key=lambda subl: (subl[0], subl[1], subl[2]))
wav_cols = ["Patient number", "Recording index", "Chest location", "WAV"]
df_wav_rec = pd.DataFrame(l_wav_rec, columns=wav_cols)

In [ ]:
input_wavs = np.empty((0, max_len, 1))
for rec in l_wav_rec:
    wav= rec[-1]
#     reshaped_wav = [[value] for value in wav]
    reshaped_wav = np.array(wav, dtype="float32").reshape(-1,1)
    input_wavs = np.append(input_wavs, reshaped_wav)

In [3]:
# x = np.array([[[[1.0,2.0],[1.0,2.0]],[[2.0,3.0],[2.0,3.0]]]])
# y = np.array([[[[2.0,4.0],[2.0,4.0]],[[4.0,6.0],[4.0,6.0]]]])
# x = np.array([[[[1.0]],[[2.0]]]])
# y = np.array([[[[2.0]],[[2.0]]]])
# conv = keras.layers.Conv2D(filters=1,kernel_size=2, strides=1, padding="same", activation="relu")
# model = keras.models.Sequential([conv])
# model.compile(loss="mse", metrics=["accuracy"], optimizer="sgd")
# model.fit(x, y)
# conv.get_weights()[0].shape
# conv.get_weights()[0]
#x.shape

In [4]:
# # get annotation corresponding to smallest wave file
# df_annot = pd.read_csv(fname_annot).set_index(["Patient number", "Recording index", "Chest location"])
# df_annot = df_annot.sort_index()
# df_min_annot = df_annot.loc[min_patnum, min_recindex, min_chestloc]
# df_max_annot = df_annot.loc[max_patnum, max_recindex, max_chestloc]

# max_len_annot = len(df_max_annot)

# # construct CNN target feature maps
# target_feature_maps = np.empty((0,max_len_annot,4), dtype="float32")
# for ix in df_annot.index:
#     cur_annot = df_annot.loc[ix, ["Cycle start", "Cycle end", "Crackles", "Wheezes"]]
#     # pad additional "dummy" rows to maximum length annotation
#     if len(cur_annot) < max_len_annot:
#         pad_len = max_len_annot - len(cur_annot)
#         # what's the best way of padding?
#         padding = np.array([[-1, -1, 0, 0]] * pad_len)
#         cur_annot = np.stack((cur_annot, padding), axis=1)
#         print(cur_annot)
#     feature_map = cur_annot.to_numpy()[np.newaxis, :]
# #     print(feature_map)
#     target_feature_maps = np.append(target_feature_maps, feature_map, axis=0)
# #     cur_target_feature_mapseature_map]
# target_feature_maps

In [5]:
# get annotation corresponding to smallest wave file
df_annot = pd.read_csv(fname_annot).set_index(["Patient number", "Recording index", "Chest location"])
df_annot = df_annot.sort_index()
# df_min_annot = df_annot.loc[min_patnum, min_recindex, min_chestloc]
# df_max_annot = df_annot.loc[max_patnum, max_recindex, max_chestloc]

max_len_annot = 0
max_ix = None

for ix in df_annot.index:
    cur_annot = df_annot.loc[ix, ["Cycle start", "Cycle end", "Crackles", "Wheezes"]]
    # pad additional "dummy" rows to maximum length annotation
    if len(cur_annot) > max_len_annot:
        max_len_annot = len(cur_annot)
        df_max_annot = cur_annot
#         max_ix = ix

# construct CNN target feature maps
target_feature_maps = np.empty((0,max_len_annot,4), dtype="float32")
for ix in df_annot.index:
    cur_annot = df_annot.loc[ix, ["Cycle start", "Cycle end", "Crackles", "Wheezes"]]
    # pad additional "dummy" rows to maximum length annotation
    if len(cur_annot) < max_len_annot:
        pad_len = max_len_annot - len(cur_annot)
        # what's the best way of padding?
        padding = np.array([[-1, -1, 0, 0]] * pad_len)
        cur_annot = np.vstack((cur_annot, padding))
        print(cur_annot)
    feature_map = cur_annot[np.newaxis, :]
#     print(feature_map)
    target_feature_maps = np.append(target_feature_maps, feature_map, axis=0)
#     cur_target_feature_mapseature_map]
target_feature_maps

[[ 0.036  0.579  0.     0.   ]
 [ 0.579  2.45   0.     0.   ]
 [ 2.45   3.893  0.     0.   ]
 [ 3.893  5.793  0.     0.   ]
 [ 5.793  7.521  0.     0.   ]
 [ 7.521  9.279  0.     0.   ]
 [ 9.279 11.15   0.     0.   ]
 [11.15  13.036  0.     0.   ]
 [13.036 14.721  0.     0.   ]
 [14.721 16.707  0.     0.   ]
 [16.707 18.507  0.     0.   ]
 [18.507 19.964  0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.   

[[ 0.       0.54469  0.       1.     ]
 [ 0.54469  2.9628   0.       1.     ]
 [ 2.9628   5.1085   0.       1.     ]
 [ 5.1085   7.2172   0.       1.     ]
 [ 7.2172   9.1442   0.       1.     ]
 [ 9.1442  10.675    0.       1.     ]
 [10.675   12.371    0.       1.     ]
 [12.371   14.381    0.       1.     ]
 [14.381   16.535    0.       1.     ]
 [16.535   19.048    0.       1.     ]
 [19.048   22.176    0.       0.     ]
 [22.176   22.951    0.       0.     ]
 [22.951   24.664    0.       0.     ]
 [24.664   25.584    0.       0.     ]
 [-1.      -1.       0.       0.     ]
 [-1.      -1.       0.       0.     ]
 [-1.      -1.       0.       0.     ]
 [-1.      -1.       0.       0.     ]
 [-1.      -1.       0.       0.     ]
 [-1.      -1.       0.       0.     ]
 [-1.      -1.       0.       0.     ]
 [-1.      -1.       0.       0.     ]
 [-1.      -1.       0.       0.     ]
 [-1.      -1.       0.       0.     ]
 [-1.      -1.       0.       0.     ]
 [-1.      -1.       0.  

[[ 0.036  2.279  0.     0.   ]
 [ 2.279  4.879  0.     0.   ]
 [ 4.879  7.507  0.     0.   ]
 [ 7.507 10.336  0.     0.   ]
 [10.336 13.364  0.     0.   ]
 [13.364 16.179  0.     0.   ]
 [16.179 19.007  0.     0.   ]
 [19.007 19.907  0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.   

[[ 0.887  2.601  0.     0.   ]
 [ 2.601  5.327  1.     0.   ]
 [ 5.327  7.923  1.     0.   ]
 [ 7.923 10.53   1.     0.   ]
 [10.53  13.196  1.     0.   ]
 [13.196 15.565  1.     0.   ]
 [15.565 18.054  1.     0.   ]
 [18.054 19.101  1.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.   

[[ 1.018  3.411  1.     0.   ]
 [ 3.411  5.827  1.     0.   ]
 [ 5.827  8.339  1.     0.   ]
 [ 8.339 10.923  1.     0.   ]
 [10.923 13.292  1.     0.   ]
 [13.292 16.018  1.     0.   ]
 [16.018 18.482  1.     0.   ]
 [18.482 19.542  1.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.   

...

[[ 1.3445  3.0144  0.      0.    ]
 [ 3.0144  4.7622  0.      0.    ]
 [ 4.7622  6.7789  0.      0.    ]
 [ 6.7789  8.9301  0.      0.    ]
 [ 8.9301 10.975   0.      0.    ]
 [10.975  12.185   0.      0.    ]
 [12.185  13.933   0.      0.    ]
 [13.933  16.183   0.      0.    ]
 [16.183  21.936   0.      0.    ]
 [-1.     -1.      0.      0.    ]
 [-1.     -1.      0.      0.    ]
 [-1.     -1.      0.      0.    ]
 [-1.     -1.      0.      0.    ]
 [-1.     -1.      0.      0.    ]
 [-1.     -1.      0.      0.    ]
 [-1.     -1.      0.      0.    ]
 [-1.     -1.      0.      0.    ]
 [-1.     -1.      0.      0.    ]
 [-1.     -1.      0.      0.    ]
 [-1.     -1.      0.      0.    ]
 [-1.     -1.      0.      0.    ]
 [-1.     -1.      0.      0.    ]
 [-1.     -1.      0.      0.    ]
 [-1.     -1.      0.      0.    ]
 [-1.     -1.      0.      0.    ]
 [-1.     -1.      0.      0.    ]
 [-1.     -1.      0.      0.    ]
 [-1.     -1.      0.      0.    ]
 [-1.     -1.      0

[[ 0.064  0.393  0.     0.   ]
 [ 0.393  1.236  0.     0.   ]
 [ 1.236  2.193  0.     0.   ]
 [ 2.193  2.979  0.     0.   ]
 [ 2.979  3.922  0.     0.   ]
 [ 3.922  4.736  0.     0.   ]
 [ 4.736  5.664  0.     0.   ]
 [ 5.664  6.593  0.     0.   ]
 [ 6.593  7.393  0.     0.   ]
 [ 7.393  8.221  0.     0.   ]
 [ 8.221  9.236  0.     0.   ]
 [ 9.236 10.164  0.     0.   ]
 [10.164 10.836  0.     1.   ]
 [10.836 12.179  0.     0.   ]
 [12.179 13.007  0.     0.   ]
 [13.007 13.65   0.     0.   ]
 [13.65  14.593  0.     0.   ]
 [14.593 15.479  0.     0.   ]
 [15.479 16.321  0.     0.   ]
 [16.321 17.079  0.     0.   ]
 [17.079 17.879  0.     0.   ]
 [17.879 18.707  0.     0.   ]
 [18.707 19.55   0.     0.   ]
 [19.55  19.893  0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.    -1.     0.     0.   ]
 [-1.   

[[ 0.91168  3.6586   0.       1.     ]
 [ 3.6586   4.9432   0.       1.     ]
 [ 4.9432   6.3048   0.       1.     ]
 [ 6.3048   7.5243   0.       1.     ]
 [ 7.5243   8.6728   0.       1.     ]
 [ 8.6728   9.8331   0.       1.     ]
 [ 9.8331  11.781    0.       1.     ]
 [11.781   12.769    0.       1.     ]
 [12.769   18.352    0.       0.     ]
 [-1.      -1.       0.       0.     ]
 [-1.      -1.       0.       0.     ]
 [-1.      -1.       0.       0.     ]
 [-1.      -1.       0.       0.     ]
 [-1.      -1.       0.       0.     ]
 [-1.      -1.       0.       0.     ]
 [-1.      -1.       0.       0.     ]
 [-1.      -1.       0.       0.     ]
 [-1.      -1.       0.       0.     ]
 [-1.      -1.       0.       0.     ]
 [-1.      -1.       0.       0.     ]
 [-1.      -1.       0.       0.     ]
 [-1.      -1.       0.       0.     ]
 [-1.      -1.       0.       0.     ]
 [-1.      -1.       0.       0.     ]
 [-1.      -1.       0.       0.     ]
 [-1.      -1.       0.  

[[ 0.89359  3.4344   0.       0.     ]
 [ 3.4344   6.0183   0.       0.     ]
 [ 6.0183   8.9467   0.       0.     ]
 [ 8.9467  11.337    0.       0.     ]
 [11.337   16.612    0.       0.     ]
 [16.612   19.121    0.       0.     ]
 [19.121   21.834    0.       0.     ]
 [21.834   24.848    0.       0.     ]
 [24.848   27.594    0.       0.     ]
 [27.594   30.221    0.       0.     ]
 [30.221   32.826    0.       0.     ]
 [32.826   34.721    0.       0.     ]
 [34.721   37.972    0.       0.     ]
 [37.972   43.538    0.       0.     ]
 [43.538   49.018    0.       0.     ]
 [49.018   54.95     0.       0.     ]
 [54.95    60.344    0.       0.     ]
 [60.344   63.779    0.       0.     ]
 [63.779   66.75     0.       0.     ]
 [-1.      -1.       0.       0.     ]
 [-1.      -1.       0.       0.     ]
 [-1.      -1.       0.       0.     ]
 [-1.      -1.       0.       0.     ]
 [-1.      -1.       0.       0.     ]
 [-1.      -1.       0.       0.     ]
 [-1.      -1.       0.  

TypeError: '(None, slice(None, None, None))' is an invalid key

In [ ]:
max_len
# max_len_annot

In [ ]:
# x1 = np.array([[df_wav_rec.loc[0, "WAV"].astype("float32")]])
x1 = np.array([df_wav_rec.loc[0, "WAV"].astype("float32")]).reshape(-1,1)[np.newaxis,:,:]
conv = keras.layers.Conv1D(filters=32,kernel_size=5, strides=2, padding="same", activation="relu")

cnn_strides = [1,2,4]
fmap_size = max_len

model = keras.models.Sequential()
for i in range(len(cnn_strides)):
    model.add(keras.layers.Conv1D(filters = 64 / (2 ** i) , kernel_size= 5 * (2 ** i), strides=cnn_strides[i], padding="same", activation="relu"))
    fmap_size //= cnn_strides[i]

# calculate this in order to get the last stride number that results in a output feature map of
# size of longest rec. annotation
last_strides = fmap_size // max_len_annot

model.add(keras.layers.Conv1D(filters = 1 , kernel_size= 5 * (2 ** (i+1)), strides=last_strides, padding="same", activation="relu"))

model.compile(loss="mse", metrics=["accuracy"], optimizer="sgd")

input_wav = []

# model.fit(l_wav_rec, target_feature_maps)
# model.fit(x2, y)

In [ ]:
# np.fromiter((), dtype="float32")
[ [] for wav in l_wav_rec:]